In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://i.imgur.com/RhSYbDW.png' height="100" alt="netdata"/></center>
#@markdown <center><h3>☁️ Cloud Torrent: a self-hosted remote torrent client</h3></center><br>
import os, sys, subprocess; from IPython.display import clear_output

!pkill ssh

if os.path.isfile("/usr/local/bin/cloud-torrent") == False:
  os.system('curl https://i.jpillora.com/cloud-torrent! | bash')
else:
  !pkill cloud-torrent
clear_output(wait=True)

#MEGAcmd installing
if os.path.isfile("/usr/bin/mega-cmd") == False:
  os.system('sudo apt-get -y update && sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https && sudo curl -sL -o/var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb && sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb && sudo ldconfig')
else:
  !pkill mega-cmd
clear_output(wait=True)

#Google Drive CLI Client
if os.path.isfile("/usr/local/bin/gdrive") == False:
  !wget -q -c -nc https://github.com/gdrive-org/gdrive/releases/download/2.1.0/gdrive-linux-x64
  !mv gdrive-linux-x64 /usr/local/bin/gdrive
  !sudo chmod +x /usr/local/bin/gdrive
  !rm -f /content/gdrive-linux-x64
  clear_output(wait=True)
else:
  !pkill gdrive

#Forwarding public server by serveo.net
!nohup ssh -o StrictHostKeyChecking=no -R 80:localhost:4444 serveo.net > link.out &

#Opening cloud-torrent in background
!nohup cloud-torrent --port 4444 --title "Cloud Torrent" --log &
clear_output(wait=True)
print(os.popen('tail -n 2 link.out').read())


In [0]:
#@markdown <br><center><img src='https://cdn0.iconfinder.com/data/icons/mattress-1/512/mattress-pad-bed-01-512.png' height="100" alt="netdata"/></center>
#@markdown <center><h2>Extracte Zip</h2></center><br>


!cd downloads/ && find . -name '*.zip' -exec sh -c 'unzip -d "`dirname \"{}\"`" "{}" && sudo rm "`dirname \"{}\"`" "{}"' ';'

![alt text](https://mega.nz/favicon.ico?v=3) 
#TRANSFERE to MEGA

---





In [0]:
#@markdown <center><h2>INPUT YOUR MEGA ID</h2></center><br>
#@markdown <center>(If you see Fetching nodes 100% on stuck, rerun)</center><br>

import os
import contextlib
import subprocess
import time

#INPUT YOUR MEGA ID

USERNAME = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
!sudo ldconfig
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def example():
    cmd = ['mega-login', USERNAME, PASSWORD]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)
example()
!mega-whoami


In [0]:
#@markdown <br><center><img src='https://www.impactcomputing.co.uk/uploads/files/services/managed-office-365.svg' height="100" alt="netdata"/></center>
#@markdown <center><h2>TRANSFERE NOW</h2></center><br>
#TRANSFERE NOW

import contextlib
import subprocess
import time


# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def example():
    cmd = ['mega-put', 'downloads', '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

example()

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ad/Logo_of_Google_Drive.svg" weight="100px" height="100px" />
<h2>TRANSFERE to Google drive</h2>

In [0]:
#@markdown <h3>⬅️ Click Here to</h3>

#@markdown #INPUT YOUR GOOGLE DRIVE authtoken
#@markdown #### (SOMETIMES WRITING ISSUES. SO RUN AGAIN)

!gdrive about


In [0]:
#UPLOAD TO GOOGLE DRIVE
#@markdown <br><center><img src='https://support.questetra.com/wp-content/uploads/2018/04/googledrive-simpleupload-fi-en.png' height="200" alt="netdata"/></center>
#@markdown <center><h2>TRANSFERE NOW</h2></center><br>

!gdrive upload --recursive /content/downloads